### Librerias necesarias

In [14]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#### Cargo la tabla ordesrs y las ventas históricas

In [15]:
orders = pd.read_csv("./archivos_csv/orders.csv")
ventas_df = pd.read_csv("./archivos_csv/ventas_historico.csv")

#### Transformo la tabla ventas historicas para tener todos los productos con sus respectivos pedidos

In [16]:
ventasxproducts_parcial = ventas_df[["orderId", "products"]]
ventasxproducts_parcial.drop_duplicates(subset=['orderId'], inplace=True)
ventasxproducts_parcial['products'] = ventasxproducts_parcial['products'].apply(lambda x: eval(x))

expanded_rows = []
# Iterar sobre cada fila del DataFrame original
for index, row in ventasxproducts_parcial.iterrows():
    order_id = row['orderId']
    products = row['products']
    # Iterar sobre cada producto en la lista de productos
    for product in products:
        # Crear un nuevo diccionario que representa una fila expandida
        expanded_row = {**product, 'orderId': order_id}
        expanded_rows.append(expanded_row)

# Crear un nuevo DataFrame con las filas expandidas
df_expanded = pd.DataFrame(expanded_rows)

df_expanded

,payed,discounts,hierarchyName,netPrice,name,hierarchyId,id,taxes,quantity,orderId
0,6990.0,0.0,Burgers🍔,6990.0,Hamburguesa Americana🥓,AB.410,9050,1116.0,1,4617000165572608
1,3213.0,0.0,Todos los Productos,3213.0,Costo Delivery,AB.,TOTEATDVYCOST,513.0,1,4617000165572608
2,5990.0,0.0,Fries,5990.0,Mexa Grande,AB.010,1090,956.0,1,4878437089804288
3,5990.0,0.0,Fries,5990.0,Cheese and bacon Grande,AB.010,1050,956.0,1,4878437089804288
4,5990.0,0.0,Fries,5990.0,Pollo albahaca grande,AB.010,1920,956.0,1,4878437089804288
...,...,...,...,...,...,...,...,...,...,...
223409,0.0,0.0,Elige tus papas grandes,0.0,Mexa grande eats,BA.330,2340,0.0,1,6740702414372864
223410,0.0,0.0,Bar,0.0,Mojitos,AB.075,3480,0.0,1,6750384411508736
223411,13380.0,0.0,Bar,6690.0,Mojito coco,AB.075,7450,2136.0,2,6750384411508736
223412,0.0,0.0,PAPITAS,0.0,Cheese & bacon,AB.530,4370,0.0,1,6750384411508736


In [17]:
df_1 = df_expanded
df_1.rename(columns={'orderId': 'order_id'}, inplace=True)
df_1.head(3)

,payed,discounts,hierarchyName,netPrice,name,hierarchyId,id,taxes,quantity,order_id
0,6990.0,0.0,Burgers🍔,6990.0,Hamburguesa Americana🥓,AB.410,9050,1116.0,1,4617000165572608
1,3213.0,0.0,Todos los Productos,3213.0,Costo Delivery,AB.,TOTEATDVYCOST,513.0,1,4617000165572608
2,5990.0,0.0,Fries,5990.0,Mexa Grande,AB.010,1090,956.0,1,4878437089804288


#### Cargo la tabla de formas de pago para saber que precio va a tener cada producto según su forma de pago

In [18]:
df_2 = pd.read_csv("./archivos_csv/forma_pago.csv")

#### Uno ambas tablas

In [19]:
merged_df = pd.merge(df_1, df_2, on='order_id', how='inner')
merged_df.head()

,payed,discounts,hierarchyName,netPrice,name,hierarchyId,id,taxes,quantity,order_id,payment_id,amount
0,6990.0,0.0,Burgers🍔,6990.0,Hamburguesa Americana🥓,AB.410,9050,1116.0,1,4617000165572608,5000,10203.0
1,3213.0,0.0,Todos los Productos,3213.0,Costo Delivery,AB.,TOTEATDVYCOST,513.0,1,4617000165572608,5000,10203.0
2,5990.0,0.0,Fries,5990.0,Mexa Grande,AB.010,1090,956.0,1,4878437089804288,50002,17970.0
3,5990.0,0.0,Fries,5990.0,Cheese and bacon Grande,AB.010,1050,956.0,1,4878437089804288,50002,17970.0
4,5990.0,0.0,Fries,5990.0,Pollo albahaca grande,AB.010,1920,956.0,1,4878437089804288,50002,17970.0


#### Ahora uno la tabla merged con las fechas

In [20]:
# Primero cambio el nombre de la columna id en orders
orders.rename(columns={'id': 'order_id'}, inplace=True)
# Ahora las uno
df_final = pd.merge(merged_df, orders, on='order_id', how='inner')
df_final.head()

,payed,discounts,hierarchyName,netPrice,name,hierarchyId,id,taxes,quantity,order_id,payment_id,amount,date_open,date_closed
0,6990.0,0.0,Burgers🍔,6990.0,Hamburguesa Americana🥓,AB.410,9050,1116.0,1,4617000165572608,5000,10203.0,2021-08-14T00:55:44,2021-08-14T01:50:47
1,3213.0,0.0,Todos los Productos,3213.0,Costo Delivery,AB.,TOTEATDVYCOST,513.0,1,4617000165572608,5000,10203.0,2021-08-14T00:55:44,2021-08-14T01:50:47
2,5990.0,0.0,Fries,5990.0,Mexa Grande,AB.010,1090,956.0,1,4878437089804288,50002,17970.0,2021-08-14T01:26:35,2021-08-14T20:23:29
3,5990.0,0.0,Fries,5990.0,Cheese and bacon Grande,AB.010,1050,956.0,1,4878437089804288,50002,17970.0,2021-08-14T01:26:35,2021-08-14T20:23:29
4,5990.0,0.0,Fries,5990.0,Pollo albahaca grande,AB.010,1920,956.0,1,4878437089804288,50002,17970.0,2021-08-14T01:26:35,2021-08-14T20:23:29


In [21]:
# Elimino las columnas que no necesito
df_final = df_final.drop(columns=['payed', 'amount', 'date_open'])

# Solo tomo los registros que no tiene descuentos, ya que los descuentos afectan al precio neto y este
# es el que tomamos como referencia. También solo los que tiene cantidad 1 porque la cantidad también
# afecta al precio neto.
df_final = df_final[(df_final['discounts'] == 0) & (df_final['quantity'] == 1)]

# Ordeno los registros
df_final = df_final.sort_values(by=['id', 'payment_id', 'date_closed'])

# Vemos los primeros 50 registros para ver el formato de la tabla.
df_final.head(50)

,discounts,hierarchyName,netPrice,name,hierarchyId,id,taxes,quantity,order_id,payment_id,date_closed
118288,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,6247356571582464,1000,2022-12-23T01:58:00
119588,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,6310175300124672,1000,2022-12-28T20:58:24
117565,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,6149415278215168,3000,2022-12-19T00:11:32
118289,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,6247356571582464,3000,2022-12-23T01:58:00
118695,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,6021096285143040,3000,2022-12-24T01:11:19
121563,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,5437771979948032,3000,2023-01-06T00:11:12
89130,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,5619266654830592,5119,2022-09-10T01:59:08
89658,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,5800403343769600,5119,2022-09-11T01:24:28
89980,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,5896762646331392,5119,2022-09-11T17:29:07
90874,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,5119539928629248,5119,2022-09-16T00:45:13


#### Eliminamos los duplicados

In [22]:
df_sin_duplicados = df_final.drop_duplicates(subset=['id', 'payment_id', 'netPrice'])
df_sin_duplicados.head(20)

,discounts,hierarchyName,netPrice,name,hierarchyId,id,taxes,quantity,order_id,payment_id,date_closed
118288,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,6247356571582464,1000,2022-12-23T01:58:00
117565,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,6149415278215168,3000,2022-12-19T00:11:32
89130,0.0,Favoritos🔥,14990.0,2 Burger + Papas Grandes,AB.420,10,2393.0,1,5619266654830592,5119,2022-09-10T01:59:08
42,0.0,Fries,2990.0,Single fries Standard,AB.010,1000,477.0,1,4627544629051392,1000,2021-08-14T22:58:34
39532,0.0,Fries,3990.0,Single fries Standard,AB.010,1000,637.0,1,5585166360379392,1000,2022-02-14T18:21:32
51863,0.0,Fries,3490.0,Single fries Standard,AB.010,1000,557.0,1,5774259613597696,1000,2022-04-02T22:03:15
71060,0.0,Fries,6490.0,Single fries Standard,AB.010,1000,1036.0,1,5182969171673088,1000,2022-06-30T20:02:11
71873,0.0,Fries,6490.0,Single fries Standard,AB.010,1000,1036.0,1,5856462188052480,2000,2022-07-03T03:08:22
135941,0.0,Fries,3490.0,Single fries Standard,AB.010,1000,557.0,1,6564665991364608,2000,2023-02-19T02:08:24
258,0.0,Fries,2990.0,Single fries Standard,AB.010,1000,477.0,1,5372787255148544,3000,2021-08-23T22:37:40


Conclusiones hasta el momento:
- Existen varios precios para un mismo producto y metodo de pago.
- No necesariamente significa aumentos con el tiempo porque a veces incluso bajan de precio, o se duplican de unos meses a otro.
- Posible solucion: introducir los precios manualmente y que, en un futuro, al momento de tomar un pedido ese precio se cargue en la tabla sold products
(en una columan que deberiamos crear y se llame precio actual, por ejemplo).